# Data-Driven Semantic Segmentation Pipeline for Seismic Faults

## Introduction: 

Seismic data acts as the Earth's MRI, providing foundational information for geologists studying the planet's structure. Among various research topics, seismic fault segmentation stands out as a crucial area in geological studies. In the current competition, the organizers provided only 400 training samples, some of which contain erroneous labels, categorizing this task as a small-sample, noisy semantic segmentation problem. To tackle this challenge, we implemented a data-driven pipeline design approach and developed the first 3D segmentation pipeline specifically tailored for seismic data. By leveraging a substantial amount of unlabeled data from previous competition training sets and newly generated data from the official data generator, we significantly enhanced the model's prediction accuracy despite the limited availability of labeled data.

## Methodology:

### 1. Erroneous Samples in the Training Set

We initiated the process by training the model on the officially provided labeled dataset and performed 5-fold cross-validation. Upon analyzing the prediction results on the validation set, we identified several issues:

__Detection of Faulty Labels:__ Analysis of the out-of-fold predictions revealed that a few samples had Dice coefficients of 0, indicating complete misalignment. Further investigation showed that 12 samples in the training set had incorrect labels. The following images illustrate examples of these mislabeled samples. We removed these erroneous annotations from the training set and treated them as unlabeled data.

![aaa](./imgs/err_crop.png)

### 2. Insufficient Diversity in the Training Set

Continuing the analysis of low-scoring samples in the out-of-fold predictions, we identified six samples containing salt bodies. These instances were exceedingly rare in the dataset, leading the model to fail in appropriately handling salt bodies, often misclassifying them as faults and resulting in numerous false positives. To address this, we modified the data generation pipeline provided by Onward to produce additional samples containing salt bodies for training. It is important to note that to ensure the openness and accessibility of the publicly available dataset, we made the modified data generation scripts open-source from the outset (available at github.com/***). Additionally, to prevent unfair advantages from using extra labeled data, we adjusted the generator to exclude fault labels and treated the newly generated seismic data as unlabeled data for training. The augmented data distribution is illustrated in the figure below.

![aaa](./imgs/bar.png)

### 3. Extremely Low Proportion of Positive Labels

Upon examining the training set, we observed that faults occupy minimal spatial volume. Almost all faults manifest as thin slices with a thickness of only 1-2 pixels in both the inline and crossline directions. This means that any scaling or interpolation of labels, whether by integer multiples or not, results in complete misalignment of the fault slices. Experiments demonstrated that misaligned predictions after interpolation led to a Dice score reduction of 0.2 to 0.4, depending on the pixel offset, even though the differences are barely noticeable to the naked eye. Consequently, we opted for a convolution-based 3D UNet semantic segmentation network, allowing the network to directly incorporate global information for voxel-wise predictions. Compared to 2D networks operating on crossline and inline directions, end-to-end 3D training yielded significantly more stable results. However, the large input dimensions (300, 300, 1259) were too substantial for direct network input. Without the option to downsample, we employed three-dimensional random window sampling. Extensive experiments led us to select a patch size of 192 for the crossline and inline directions and 640 for the TWT (Two-Way Travel Time) dimension. This sampling strategy, covering more than half the volume, not only reduced GPU memory requirements but also provided sufficient contextual information.

### 4. Difficulty in Distinguishing Fault Edges

Due to the nature of faults, a fault occurring in the inline direction presents as distinct interlayer overlaps, whereas in the crossline direction, it appears as a wide band. This band manifests as a blurred mass with edges that are challenging to discern, leading to low-confidence predictions and unstable scores at the boundaries. The official starter scripts suggested that frequency domain decomposition might be beneficial, hinting at periodic characteristics in the data. Upon analysis, we confirmed that the data exhibits clear periodicity in the TWT dimension, characterized by the superimposition of multiple sine waves, as shown in the figure below.

Therefore, frequency-based filtering holds significant potential for enhancing fault visibility. However, frequency decomposition is computationally intensive, taking approximately one hour per sample on a 64-core 4.0 GHz Threadripper processor. To optimize computational efficiency, we redesigned the feature engineering pipeline. Instead of frequency decomposition, we employed gradient calculations and utilized a simple np.roll operation to identify zero-crossings and extrema in the TWT dimension. This operation generated two binary channels—zero crossings and extrema—which exhibited distinct changes and sharp edges at fault locations. Experiments demonstrated that incorporating these two channels improved both convergence speed and final accuracy. Ultimately, our input consisted of four channels: the original seismic wave, the gradient channel, zero crossings of the seismic wave, and extrema of the seismic wave (gradient zeros).

![aaa](./imgs/4c_box.png)

### 5. Overfitting Due to Limited Annotations

The primary challenge of this competition was to train a robust segmentation model with a limited amount of data. To mitigate overfitting, we designed multiple data augmentation strategies tailored to the characteristics of seismic data, as illustrated in the diagram below. Leveraging the symmetry of seismic data and the similarity between crossline and inline directions, we applied random flips along the crossline and inline axes and performed random transpositions on these two dimensions during training. This approach resulted in an eightfold increase in data augmentation, significantly reducing the risk of overfitting. Additionally, during inference on the test set, we applied eight combinations of data augmentations to further enhance prediction stability.

Furthermore, we utilized the best-performing fully supervised model to generate pseudo-labels for a large amount of unlabeled data. By training the model jointly on both pseudo-labels and true labels, we improved the model's generalization performance and enhanced prediction stability on datasets containing salt bodies.

![aaa](./imgs/aug.png)

## Reproduction Guidelines:

__Environment Setup:__

Begin by installing the required packages in a Conda environment using the following command:

```
conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia
pip install segmentation_models_pytorch_3d tqdm
```

__Training:__

Generating additional data takes approximately one month on a 64-core 4.0 GHz Threadripper processor.

The training script is train.py, which should be executed on an 8x4090 GPU server. Depending on I/O performance, training may take between one to two weeks.

__Pre-trained Models:__

To expedite reproduction, we provide pre-trained model checkpoints. To run the inference directly, please follow the instructions outlined in inference.ipynb.

__Additional Notes:__

The modified data generation scripts are available on GitHub at https://github.com/
Ensure that all dependencies are correctly installed to avoid compatibility issues.
For any issues or further assistance, please feel free to contact us.